# Pre-process the dataset for FelineFlow
The original Cats vs. Dogs dataset consists of more than 12,000 images, each of different sizes of cats alone! We need to modify the dataset to suit our needs accordingly.

In this notebook, we will apply the following transformations:
- Select 4096 images for training and testing.
- Crop the images to a 1:1 aspect ratio.
- Check if the images have a single cat, in frame using Haar Cascades (and choose another image otherwise)
- Downscale the image to 128x128.

Let's get started by importing the necessary modules

In [135]:
import os
import cv2

Let's create a function to crop an image to 1:1

In [136]:
def crop_square(img_path):
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    if(height==width):
        return img
    if(height>width):
        return img[(height-width)//2: (height+width)//2, :]
    else:
        return img[:, (width-height)//2: (width+height)//2]

Function to resize the image to a specified resolution

In [137]:
def resize(img, res):
    return cv2.resize(img, res, interpolation=cv2.INTER_AREA)

A module that recognises cat faces. We will be using the [Cat Frontal Face Haar Cascade](https://github.com/opencv/opencv/blob/master/data/haarcascades/haarcascade_frontalcatface_extended.xml) contributed by Joseph Howse provided on the OpenCV GitHub repository

In [138]:
def isValidCat(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cascade = cv2.CascadeClassifier('./haarcascade_frontalcatface.xml')
    faces = cascade.detectMultiScale(gray, 1.1, 3) #image, reject levels, level weight
    return len(faces)==1

In [139]:
isValidCat(crop_square('./cats_source/122.jpg'))

False

And finally, a function to generate the dataset

In [142]:
def generate_dataset(input_dir, output_dir, res, num_images):
    os.makedirs(output_dir, exist_ok=True)
    
    input_files = os.listdir(input_dir)
    img_len = len(input_files)
    if(img_len<num_images):
        raise Exception("Not enough images in source directory")
    count=1
    for file in input_files:
        if(count>=num_images):
            break
        try:
            destination_path = os.path.join(output_dir, str(count))+'.jpg'
            cropped_image = crop_square(os.path.join(input_dir, file))
            if(not isValidCat(cropped_image)):
                continue
                            
            resized_image = resize(cropped_image, res)
            if(count%128==0):
                print(count, " images processed")
            cv2.imwrite(destination_path, resized_image)
            count+=1
        except AttributeError:
            continue    

Let's specify the parameters globally and watch the script work its magic!

In [143]:
generate_dataset('./cats_source', './cats_processed', (256,256), 4096)

128  images processed
256  images processed
384  images processed
512  images processed
640  images processed
768  images processed
896  images processed
1024  images processed
1152  images processed
1280  images processed
1408  images processed
1536  images processed
1664  images processed
1792  images processed
1920  images processed
2048  images processed
2176  images processed
2304  images processed
2432  images processed
